In [ ]:
# If the libraries are not yet installed, they can be installed using conda commands similar to the below
# %conda install pandas
# %conda install requests 

# Something like the following may also work if the above does not
# import sys
# !conda install --yes --prefix {sys.prefix} pandas
# !conda install --yes --prefix {sys.prefix} requests

# To install a specific version, add the version to the install command
# E.g., %conda install pandas=1.3.4

# If all else fails, use pip or follow additional advice such as found at
# https://jakevdp.github.io/blog/2017/12/05/installing-python-packages-from-jupyter/

# If your plan to use pip (especially if you are not working within a specified conda environment), 
# the pip commands might look like:
# pip install pandas
# pip install requests

# To install a specific version, add the version to the pip install command
# E.g., pip install pandas==1.3.4

In [ ]:
# Load necessary libraries
import requests
import pandas as pd
from io import StringIO
import time
import datetime
import os

In [ ]:
os.makedirs("Data/Raw/Highways_England", exist_ok=True)

os.makedirs("Data/Raw/Portland", exist_ok=True)

os.makedirs("Data/Raw/Utah", exist_ok=True)

In [ ]:
def get_portland_vancouver_data(date_list, highway_id="6", resolution="00:15:00", detector_choice=None):
    """Function to get portland-vancouver regional area highway data from the provided api"""
    
    # Set API headers
    headers = {
        'Host': 'new.portal.its.pdx.edu:8080',
        'Referer': 'http://new.portal.its.pdx.edu:8080/downloads/'
    }
    
    # Parameters for the while loop, which limits retries upon failure,
    # and for the counter, which loops through the full date range provided in the function call
    n = len(date_list)-1
    counter = 0
    
    # Initialize empty data frame
    full_df = pd.DataFrame()
    

    while counter < n:
        print(f"Reading for date range {date_list[counter]} to {date_list[counter+1]}")

        # Make API calls within try-except block to catch failures
        try:
            start = time.time()

            # Set params for API call
            params = (
                ('start_date', date_list[counter]),
                ('end_date', date_list[counter+1]),
                ('highway_id', [highway_id]),
                ('resolution', resolution),
                ('format', 'csv')
            )

            # Make API call
            response = requests.get(
                'http://new.portal.its.pdx.edu:8080/highways/api/freewaydata/',
                headers = headers, params = params)

            end = time.time()
            print(f"API Call took {round(end-start, 2)} seconds")

            # Check that the call was successful, if not raise a generic exception
            state = response.status_code
            if state != 200:
                raise Exception(state)

            # If the call was successful, get the text body from the result and read in to df
            df = pd.read_csv(StringIO(response.text))

            # If a detector is specifed in function call, filter to only that detector in result df
            if detector_choice:    
                df = df.query("detector_id in @detector_choice")

            # Append to full data frame
            full_df = full_df.append(df)

            # Increment the counter
            counter += 1

        except Exception as err:
            print(f"API call failed. Retrying in 2 seconds.")

        # Sleep 2 seconds between API calls
        time.sleep(2)

        print("-------------------------")
            

    # Return dataframe with full results
    return full_df

In [ ]:
# Create a date list which breaks up. the year 2019 into 5-day long intervals to use with the API calling function
date_list = ['2018-12-31', 
             '2019-01-05', 
             '2019-01-10',
             '2019-01-15',
             '2019-01-20',
             '2019-01-25',
             '2019-01-31',
             '2019-02-01',
             '2019-02-05',
             '2019-02-10',
             '2019-02-15',
             '2019-02-20',
             '2019-02-28',
             '2019-03-01',
             '2019-03-05',
             '2019-03-10',
             '2019-03-15',
             '2019-03-20',
             '2019-03-25',
             '2019-03-31',
             '2019-04-01',
             '2019-04-05',
             '2019-04-10',
             '2019-04-15',
             '2019-04-20',
             '2019-04-25',
             '2019-04-30',
             '2019-05-01',
             '2019-05-05',
             '2019-05-10',
             '2019-05-15',
             '2019-05-20',
             '2019-05-25',
             '2019-05-31',
             '2019-06-01',
             '2019-06-05',
             '2019-06-10',
             '2019-06-15',
             '2019-06-20',
             '2019-06-25',
             '2019-06-30',
             '2019-07-01',
             '2019-07-05',
             '2019-07-10',
             '2019-07-15',
             '2019-07-20',
             '2019-07-25',
             '2019-07-31',
             '2019-08-01',
             '2019-08-05',
             '2019-08-10',
             '2019-08-15',
             '2019-08-20',
             '2019-08-25',
             '2019-08-31',
             '2019-09-01',
             '2019-09-05',
             '2019-09-10',
             '2019-09-15',
             '2019-09-20',
             '2019-09-25',
             '2019-09-30',
             '2019-10-01',
             '2019-10-05',
             '2019-10-10',
             '2019-10-15',
             '2019-10-20',
             '2019-10-25',
             '2019-10-31',
             '2019-11-01',
             '2019-11-05',
             '2019-11-10',
             '2019-11-15',
             '2019-11-20',
             '2019-11-25',
             '2019-11-30',
             '2019-12-01',
             '2019-12-05',
             '2019-12-10',
             '2019-12-15',
             '2019-12-20',
             '2019-12-25',
             '2020-01-01'
]

In [ ]:
# Make the API calls for highway 1
df = get_portland_vancouver_data(date_list, highway_id="1", 
                                 resolution="00:15:00")
df.to_csv('Data/Raw/Portland/all_sensors_highway1_portland.csv', index=False)

In [ ]:
# Make the API calls for highway 2 and save the data
df = get_portland_vancouver_data(date_list, highway_id="2", 
                                 resolution="00:15:00")
df.to_csv('Data/Raw/Portland/all_sensors_highway2_portland.csv', index=False)

In [ ]:
# Make the API calls for highway 3 and save the data
df = get_portland_vancouver_data(date_list, highway_id="3", 
                                 resolution="00:15:00")
df.to_csv('Data/Raw/Portland/all_sensors_highway3_portland.csv', index=False)

In [ ]:
# Make the API calls for highway 4 and save the data
df = get_portland_vancouver_data(date_list, highway_id="4", 
                                 resolution="00:15:00")
df.to_csv('Data/Raw/Portland/all_sensors_highway4_portland.csv', index=False)

In [ ]:
# Make the API calls for highway 5 and save the data
df = get_portland_vancouver_data(date_list, highway_id="5", 
                                 resolution="00:15:00")
df.to_csv('Data/Raw/Portland/all_sensors_highway5_portland.csv', index=False)

In [ ]:
# Make the API calls for highway 6 and save the data
df = get_portland_vancouver_data(date_list, highway_id="6", 
                                 resolution="00:15:00")
df.to_csv('Data/Raw/Portland/all_sensors_highway6_portland.csv', index=False)

In [ ]:
# Make the API calls for highway 7 and save the data
df = get_portland_vancouver_data(date_list, highway_id="7", 
                                 resolution="00:15:00")
df.to_csv('Data/Raw/Portland/all_sensors_highway7_portland.csv', index=False)

In [ ]:
# Make the API calls for highway 8 and save the data
df = get_portland_vancouver_data(date_list, highway_id="8", 
                                 resolution="00:15:00")
df.to_csv('Data/Raw/Portland/all_sensors_highway8_portland.csv', index=False)

In [ ]:
# Make the API calls for highway 9 and save the data
df = get_portland_vancouver_data(date_list, highway_id="9", 
                                 resolution="00:15:00")
df.to_csv('Data/Raw/Portland/all_sensors_highway9_portland.csv', index=False)

In [ ]:
# Make the API calls for highway 10 and save the data
df = get_portland_vancouver_data(date_list, highway_id="10", 
                                 resolution="00:15:00")
df.to_csv('Data/Raw/Portland/all_sensors_highway10_portland.csv', index=False)

In [ ]:
# Make the API calls for highway 11 and save the data
df = get_portland_vancouver_data(date_list, highway_id="11", 
                                 resolution="00:15:00")
df.to_csv('Data/Raw/Portland/all_sensors_highway11_portland.csv', index=False)

In [ ]:
# Make the API calls for highway 12 and save the data
df = get_portland_vancouver_data(date_list, highway_id="12", 
                                 resolution="00:15:00")
df.to_csv('Data/Raw/Portland/all_sensors_highway12_portland.csv', index=False)

In [ ]:
df = get_portland_vancouver_data(date_list, highway_id="50", 
                                 resolution="00:15:00")
df.to_csv('Data/Raw/Portland/all_sensors_highway50_portland.csv', index=False)

In [ ]:
df = get_portland_vancouver_data(date_list, highway_id="51", 
                                 resolution="00:15:00")
df.to_csv('Data/Raw/Portland/all_sensors_highway51_portland.csv', index=False)

In [ ]:
df = get_portland_vancouver_data(date_list, highway_id="52", 
                                 resolution="00:15:00")
df.to_csv('Data/Raw/Portland/all_sensors_highway52_portland.csv', index=False)

In [ ]:
df = get_portland_vancouver_data(date_list, highway_id="53", 
                                 resolution="00:15:00")
df.to_csv('Data/Raw/Portland/all_sensors_highway53_portland.csv', index=False)

In [ ]:
df = get_portland_vancouver_data(date_list, highway_id="618", 
                                 resolution="00:15:00")
df.to_csv('Data/Raw/Portland/all_sensors_highway618_portland.csv', index=False)

In [ ]:
df = get_portland_vancouver_data(date_list, highway_id="620", 
                                 resolution="00:15:00")
df.to_csv('Data/Raw/Portland/all_sensors_highway620_portland.csv', index=False)

In [ ]:
df = get_portland_vancouver_data(date_list, highway_id="621", 
                                 resolution="00:15:00")
df.to_csv('Data/Raw/Portland/all_sensors_highway621_portland.csv', index=False)